In [58]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from nilearn import image, plotting
from sklearn.metrics import mean_squared_error
import ptn_script as ptn

## 1. Load Data

In [59]:
#specify number of subjects to process (max 810 or just edit loader)
n_subjects = 20
traits = pd.read_csv('all_traits.csv')
neuroticism  = traits['NEOFAC_N'].to_numpy()
netmats = np.loadtxt('netmats2_clean.txt',max_rows = n_subjects)
neuroticism = neuroticism[:n_subjects]

## 2. Try Models

### 2.1 Connectome-based Predictive Modeling (CPM)
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5526681/

Takes the connectivity matrices and turns them into a one (or two) dimensional summary score based on the correlation of the edges with the behavioural metric being predicted and does a simple linear regression. Very simple method to use as baseline for prediction accuracy, but does not have much to interpret.
Model is trained and predictions done using leave one out strategy.

In [60]:
pred_CPM, edge_CPM, MSE_CPM = ptn.leave_one_out_CPM(netmats,neuroticism)

/home/annabelle/miniconda3/lib/python3.7/site-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


CPM fold:  0


/home/annabelle/miniconda3/lib/python3.7/site-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/annabelle/miniconda3/lib/python3.7/site-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/annabelle/miniconda3/lib/python3.7/site-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/annabelle/miniconda3/lib/python3.7/site-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/annabelle/miniconda3/lib/python3.7/site-packages/scipy/stats/stats.py:

CPM fold:  10


/home/annabelle/miniconda3/lib/python3.7/site-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/annabelle/miniconda3/lib/python3.7/site-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/annabelle/miniconda3/lib/python3.7/site-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/annabelle/miniconda3/lib/python3.7/site-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/annabelle/miniconda3/lib/python3.7/site-packages/scipy/stats/stats.py:

ValueError: not enough values to unpack (expected 3, got 2)

In [ ]:
print(MSE_CPM)

### 2.2 Multiple Regression
Using exact same filtering of edges by correlation as CPM, but instead fitting multiple regression model on all edge values instead of summary score. Model is trained and predictions done using leave one out strategy.

In [ ]:
pred_LR, edge_LR, MSE_LR = ptn.leave_one_out_LR(netmats,neuroticism)

In [ ]:
print(MSE_LR)

### 2.3 SVR
Using exact same filtering of edges by correlation as CPM, but instead fitting SVR model on all edge values instead of summary score. Model is trained and predictions done using leave one out strategy.

In [ ]:
pred_SVR, edge_SVR, MSE_SVR = ptn.leave_one_out_SVR(netmats,neuroticism)

In [ ]:
print(MSE_SVR)

### Vizualize connectomes

issue: dont have labels of nodes! cannot make more meaningful plots or comparisons without edge labels

In [ ]:
# ICA map
path2 = './HCP_PTN1200_recon2/groupICA_3T_HCP1200_MSMAll/groupICA/groupICA_3T_HCP1200_MSMAll_d200.ica/{}'
#ICA_map = image.load_img(path2.format('melodic_IC_sum.nii'))
coords = plotting.find_probabilistic_atlas_cut_coords(ICA_map)

In [ ]:
#find most extreme values of scores in full 810 sample
scores_total = np.matmul(netmats,mask)
pos_scores_total = np.matmul(netmats,pos_mask)
neg_scores_total = np.matmul(netmats,neg_mask)

min_scores = scores_total.argmin()
max_scores = scores_total.argmax()

min_pos = pos_scores_total.argmin()
max_pos = pos_scores_total.argmax()

min_neg = neg_scores_total.argmin()
max_neg = neg_scores_total.argmax()

In [ ]:
mean = netmats.mean(axis=0).reshape(-1,200)

In [ ]:
#average connectome of all subjects
view = plotting.view_connectome(mean, coords, edge_cmap = 'Spectral',edge_threshold = 8)
view

In [ ]:
#connectome of subject with very high neuroticism (max negative score)
view = plotting.view_connectome(netmats[max_neg].reshape(-1,200), coords, edge_cmap = 'Spectral',edge_threshold = 8)
view

In [ ]:
#connectome of subject with low neuroticism (minimum positive score)
view = plotting.view_connectome(netmats[min_pos].reshape(-1,200), coords, edge_cmap = 'Spectral',edge_threshold = 8)
view